In [3]:
import matplotlib.pyplot as plt
import cv2
import time
from navigation import NavigationSystem
from helpers import load_images_from_folder
from semseg.util import config
import numpy as np
%matplotlib inline

import matplotlib
font = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 24}

matplotlib.rc('font', **font)


mtx = np.array([[847.788207, 0.000000, 718.645947],
[0.000000, 848.418915, 455.705157],
[0.000000, 0.000000, 1.000000]])

dist = np.array([[-0.332920, 0.089132, 0.001047, -0.002518, 0.000000]])

# Process Single Image

In [4]:
CONFIG_FILE = "cfg/outdoor_pspnet50_navigation.yaml"

cfg = config.load_cfg_from_cfg_file(CONFIG_FILE)
mtxs = np.load(cfg.perspective_transform_path)
M_ = mtxs['M']
M_inv_ = mtxs['M_inv']

nav = NavigationSystem(cfg)
img = cv2.imread("./data/outdoor1/left0417.jpg")
img = cv2.undistort(img, mtx, dist, None, mtx)
dim = (cfg.original_width, cfg.original_height)
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

# fig, ax = plt.subplots(figsize=(20, 10))
# ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.show()

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h,w,_ = img.shape
start = time.time()
path, result_img, result_top, segmented_img, drivable_img, cost_fwd, cost_obst, cost_center, cost, path_top_img, lines= nav.global_planner_step(img, None)
end = time.time()
print(end-start)

# Local Planner
robot_state = np.array([0.0,0.0,0.0])
vel_cmd,yaw_rate_cmd = nav.local_planner_step(robot_state, path)


[2022-04-04 20:34:17,945 INFO pspnet_segmentation.py line 62 3227] => loading checkpoint '/usr/src/app/pretrained_models/exp/ade20k/pspnet50/model/train_epoch_100.pth'
[2022-04-04 20:34:17,945 INFO pspnet_segmentation.py line 62 3227] => loading checkpoint '/usr/src/app/pretrained_models/exp/ade20k/pspnet50/model/train_epoch_100.pth'
[2022-04-04 20:34:17,945 INFO pspnet_segmentation.py line 62 3227] => loading checkpoint '/usr/src/app/pretrained_models/exp/ade20k/pspnet50/model/train_epoch_100.pth'
[2022-04-04 20:34:17,945 INFO pspnet_segmentation.py line 62 3227] => loading checkpoint '/usr/src/app/pretrained_models/exp/ade20k/pspnet50/model/train_epoch_100.pth'
[2022-04-04 20:34:17,945 INFO pspnet_segmentation.py line 62 3227] => loading checkpoint '/usr/src/app/pretrained_models/exp/ade20k/pspnet50/model/train_epoch_100.pth'
[2022-04-04 20:34:18,135 INFO pspnet_segmentation.py line 65 3227] => loaded checkpoint '/usr/src/app/pretrained_models/exp/ade20k/pspnet50/model/train_epoch_10

Loading Object Detection
Segmentation and Detection Models loaded, Testing the models
Imgs tested
Planner
segmentation: 2.6630728244781494 seconds
warp: 0.02073526382446289 seconds


/usr/src/app/dev_ws/src/vision/vision/helpers.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  scan_array = np.array(([scan_distances, scan_angles, scan_points])).T


costmap: 0.04187440872192383 seconds
planner_: 0.010094642639160156 seconds
3.5015082359313965
motion_control: 9.274482727050781e-05 seconds


In [ ]:
results_path = './results/'
# ========== Visual Summary ============
# Segmentation
drivable_added_img = cv2.addWeighted(img, 0.9, drivable_img, 0.8, 0)  
cv2.imwrite(results_path+'00_seg_img.png',cv2.cvtColor(drivable_added_img, cv2.COLOR_BGR2RGB))

# Perspective
drivable_added_top = cv2.warpPerspective(drivable_added_img, M_, (480, 480), flags=cv2.INTER_LINEAR)
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow((drivable_added_top))
ax.set_xlabel('Y[m]')
ax.set_ylabel('X[m]')
plt.xticks(np.arange(0, drivable_added_top.shape[0]+1,96), np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-96)/80.0,2))
plt.yticks(np.arange(0, drivable_added_top.shape[0]+1,96), np.around(np.arange((drivable_added_top.shape[0]+1), 0,-96)/90.0,1))
plt.savefig(results_path+'00_seg_top.png',dpi=300)
plt.show()

# Costmap
colormap = cv2.COLORMAP_HOT
cost_heatmap = cv2.applyColorMap(np.uint8(255*cost/np.amax(cost)), colormap)
# cost_heatmap = cv2.cvtColor(cost_heatmap, cv2.COLOR_BGR2RGB)
axes = cv2.imread("./results/axes.png")
axes_img = cost_heatmap.copy()*0
axes_img[480-62:480,(240-48-15):(240+48-15),:] = axes[0:62,:,:]
# cost_heatmap = axes_img
_,thresh1 = cv2.threshold(cv2.cvtColor(axes_img, cv2.COLOR_BGR2GRAY), 2,255,cv2.THRESH_BINARY_INV)
thresh1 = np.uint8(thresh1/255)
cost_heatmap[:,:,0]*=thresh1
cost_heatmap[:,:,1]*=thresh1
cost_heatmap[:,:,2]*=thresh1
cost_heatmap+=axes_img
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(cv2.cvtColor(cost_heatmap, cv2.COLOR_BGR2RGB))
ax.set_xlabel('Y[m]')
ax.set_ylabel('X[m]')
plt.xticks(np.arange(0, cost_heatmap.shape[0]+1,96), np.around(np.arange(cost_heatmap.shape[0]/2, -(cost_heatmap.shape[0]+1)/2,-96)/80.0,2))
plt.yticks(np.arange(0, cost_heatmap.shape[0]+1,96), np.around(np.arange((cost_heatmap.shape[0]+1), 0,-96)/90.0,1))
plt.savefig(results_path+'00_cost.png',dpi=300)
plt.show()

# Paths
cost_obst_heatmap = cv2.applyColorMap(np.uint8(255*cost_obst/np.amax(cost_obst)), colormap)
img_top = cv2.warpPerspective(img, M_, (480, 480), flags=cv2.INTER_LINEAR)
img_top = cv2.cvtColor(img_top, cv2.COLOR_BGR2RGB)
path_img = 0*cost_obst_heatmap.copy()
x_idxs = np.uint16(path[:,0]*cfg.pixel_per_meter_x+cfg.width/2)
y_idxs = -np.uint16(path[:,1]*cfg.pixel_per_meter_y-cfg.height)
for i in range(x_idxs.shape[-1]-1):
    path_img = cv2.line(path_img, (x_idxs[i],y_idxs[i]), (x_idxs[i+1],y_idxs[i+1]), (0, 255, 0), 3)  
path_top = cv2.addWeighted(cost_obst_heatmap, 1.0, path_img, 0.7, 0) 
edge_lines_img = 0*cost_obst_heatmap.copy()
lines = np.array(lines, dtype = np.int)
for line in lines:
    path_top = cv2.line(path_top, (line[0],line[1]), (line[2],line[3]), (255, 50, 50), 3)
cv2.circle(path_top, (x_idxs[cfg.look_ahead], y_idxs[cfg.look_ahead]),10, (255, 255, 255), -1)
cv2.putText(path_top, 'L', (x_idxs[cfg.look_ahead]+15, y_idxs[cfg.look_ahead]+0), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (255, 255, 255), 2, cv2.LINE_AA)
path_top_added = cv2.addWeighted(img_top, 0.8, path_top, 1.0, 0)

path_top_added[:,:,0]*=thresh1
path_top_added[:,:,1]*=thresh1
path_top_added[:,:,2]*=thresh1
path_top_added+=axes_img

fig, (ax, ax2) = plt.subplots(1,2,figsize=(20, 10),gridspec_kw={'width_ratios': [1, 1.335]})
ax.imshow(cv2.cvtColor(path_top_added, cv2.COLOR_BGR2RGB))
ax.set_xlabel('Y[m]')
ax.set_ylabel('X[m]')

ax.set_xticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
ax.set_xticklabels(np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-96)/80.0,2))
ax.set_yticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
ax.set_yticklabels(np.around(np.arange((drivable_added_top.shape[0]+1), 0,-96)/90.0,1))

path_top[:,:,0]*=thresh1
path_top[:,:,1]*=thresh1
path_top[:,:,2]*=thresh1
path_top+=axes_img
unwarped_path = cv2.warpPerspective(path_top, M_inv_, (w,h), flags=cv2.INTER_LINEAR)
unwarped_path = cv2.cvtColor(unwarped_path, cv2.COLOR_BGR2RGB)
path_added_img = cv2.addWeighted(img, 1.0, unwarped_path, 0.9, 0)
path_top2 = cv2.resize(path_top, (h,h), interpolation = cv2.INTER_AREA)    
ax2.imshow(path_added_img)
ax2.axis('off')
fig.tight_layout()
plt.savefig(results_path+'00_path2.png',dpi=300)
plt.show()

fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(cv2.cvtColor(path_top_added, cv2.COLOR_BGR2RGB))
ax.set_xlabel('Y[m]')
ax.set_ylabel('X[m]')
plt.xticks(np.arange(0, cost_obst_heatmap.shape[0]+1,96), np.around(np.arange(cost_obst_heatmap.shape[0]/2, -(cost_obst_heatmap.shape[0]+1)/2,-96)/cfg.pixel_per_meter_x,2))
plt.yticks(np.arange(0, cost_obst_heatmap.shape[0]+1,96), np.around(np.arange((cost_obst_heatmap.shape[0]+1), 0,-96)/cfg.pixel_per_meter_y,1))
plt.savefig(results_path+'00_path.png',dpi=300)
plt.show()

# ========== Cost Maps ============
cost_heatmap = cv2.applyColorMap(np.uint8(255*cost/np.amax(cost)), colormap)
cost_heatmap[:,:,0]*=thresh1
cost_heatmap[:,:,1]*=thresh1
cost_heatmap[:,:,2]*=thresh1
cost_heatmap+=axes_img
cv2.imwrite(results_path+'00_cost_total.png',cost_heatmap)

cost_fwd_heatmap = cv2.applyColorMap(np.uint8(255*cost_fwd/np.amax(cost_fwd)), colormap)
cost_fwd_heatmap[:,:,0]*=thresh1
cost_fwd_heatmap[:,:,1]*=thresh1
cost_fwd_heatmap[:,:,2]*=thresh1
cost_fwd_heatmap+=axes_img
cv2.imwrite(results_path+'00_cost_fwd.png',cost_fwd_heatmap)

cost_center_heatmap = cv2.applyColorMap(np.uint8(255*cost_center/np.amax(cost_center)), colormap)
cost_center_heatmap[:,:,0]*=thresh1
cost_center_heatmap[:,:,1]*=thresh1
cost_center_heatmap[:,:,2]*=thresh1
cost_center_heatmap+=axes_img
cv2.imwrite(results_path+'00_cost_center.png',cost_center_heatmap)

cost_obst_heatmap = cv2.applyColorMap(np.uint8(255*cost_obst/np.amax(cost_obst)), colormap)
cost_obst_heatmap[:,:,0]*=thresh1
cost_obst_heatmap[:,:,1]*=thresh1
cost_obst_heatmap[:,:,2]*=thresh1
cost_obst_heatmap+=axes_img
cv2.imwrite(results_path+'00_cost_obst.png',cost_obst_heatmap)


# cv2.imwrite(results_path+'00_cost.png',cost_heatmap)


## Multiple environments test

### Sim

In [ ]:
# Create Nav System
CONFIG_FILE = "cfg/gazebosim_fchardnet_navigation.yaml"
cfg = config.load_cfg_from_cfg_file(CONFIG_FILE)
mtxs = np.load(cfg.perspective_transform_path)
M_ = mtxs['M']
M_inv_ = mtxs['M_inv']
nav = NavigationSystem(cfg)

# List of images
img_names = [
'0310.png',
'0434.png',
'0445.png',
'0486.png',
'0590.png',
'0690.png',
'0712.png',
'0714.png',
'0773.png',
'0310.png',
'0310.png',
'0310.png',
'0310.png',
'0310.png',
'0310.png',
'0310.png'
]
sim_segmented_imgs = []
sim_top_imgs = []
sim_result_imgs= []
sim_result_top_imgs= []

for img_name in img_names:
    img = cv2.imread("./data/sim/"+img_name)
#     img = cv2.undistort(img, mtx, dist, None, mtx)
    dim = (cfg.original_width, cfg.original_height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w,_ = img.shape
    path, result_img, result_top, segmented_img, drivable_img, cost_fwd, cost_obst, cost_center, cost, path_top_img, lines= nav.global_planner_step(img, None)
    
    # ======= Obtain visualizations ===========
    # Segmented
    drivable_added_img = cv2.addWeighted(img, 0.9, drivable_img, 0.8, 0) 
    sim_segmented_imgs.append(drivable_added_img)
    
    # Top View
    drivable_added_top = cv2.warpPerspective(drivable_added_img, M_, (480, 480), flags=cv2.INTER_LINEAR)
    drivable_added_top2 = cv2.resize(drivable_added_top, (h,h), interpolation = cv2.INTER_AREA) 
    sim_top_imgs.append(drivable_added_top)

    # Results
    colormap = cv2.COLORMAP_HOT
    cost_heatmap = cv2.applyColorMap(np.uint8(255*cost/np.amax(cost)), colormap)
    cost_heatmap = cv2.cvtColor(cost_heatmap, cv2.COLOR_BGR2RGB)

    cost_fwd_heatmap = cv2.applyColorMap(np.uint8(255*cost_fwd/np.amax(cost_fwd)), colormap)
    cost_fwd_heatmap = cv2.cvtColor(cost_fwd_heatmap, cv2.COLOR_BGR2RGB)

    cost_center_heatmap = cv2.applyColorMap(np.uint8(255*cost_center/np.amax(cost_center)), colormap)
    cost_center_heatmap = cv2.cvtColor(cost_center_heatmap, cv2.COLOR_BGR2RGB)

    cost_obst_heatmap = cv2.applyColorMap(np.uint8(255*cost_obst/np.amax(cost_obst)), colormap)
    cost_obst_heatmap = cv2.cvtColor(cost_obst_heatmap, cv2.COLOR_BGR2RGB)

    # Paths
    path_img = 0*cost_heatmap.copy()
    x_idxs = np.uint16(path[:,0]*cfg.pixel_per_meter_x+cfg.width/2)
    y_idxs = -np.uint16(path[:,1]*cfg.pixel_per_meter_y-cfg.height)
    for i in range(x_idxs.shape[-1]-1):
        path_img = cv2.line(path_img, (x_idxs[i],y_idxs[i]), (x_idxs[i+1],y_idxs[i+1]), (0, 255, 0), 3)  
    path_top = cv2.addWeighted(cost_obst_heatmap, 0.5, path_img, 2.0, 0) 
    edge_lines_img = 0*cost_heatmap.copy()
    if lines is not None:
        lines = np.array(lines, dtype = np.int)
        for line in lines:
            path_top = cv2.line(path_top, (line[0],line[1]), (line[2],line[3]), (50, 50, 255), 3)
    path_top_added = cv2.addWeighted(img_top, 0.8, path_top, 1.0, 0)
    
    cv2.circle(path_top, (x_idxs[cfg.look_ahead], y_idxs[cfg.look_ahead]),10, (255, 255, 255), -1)
    cv2.putText(path_top, 'L', (x_idxs[cfg.look_ahead]+15, y_idxs[cfg.look_ahead]+0), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (255, 255, 255), 2, cv2.LINE_AA)
    
    axes = cv2.imread("./results/axes.png")
    axes_img = cost_heatmap.copy()*0
    axes_img[480-62:480,(240-48-15):(240+48-15),:] = axes[0:62,:,:]
    # cost_heatmap = axes_img
    _,thresh1 = cv2.threshold(cv2.cvtColor(axes_img, cv2.COLOR_BGR2GRAY), 2,255,cv2.THRESH_BINARY_INV)
    thresh1 = np.uint8(thresh1/255)
    path_top[:,:,0]*=thresh1
    path_top[:,:,1]*=thresh1
    path_top[:,:,2]*=thresh1
    path_top+=cv2.cvtColor(axes_img, cv2.COLOR_BGR2RGB)

    
    path_top_added[:,:,0]*=thresh1
    path_top_added[:,:,1]*=thresh1
    path_top_added[:,:,2]*=thresh1
    path_top_added+=cv2.cvtColor(axes_img, cv2.COLOR_BGR2RGB)

    unwarped_path = cv2.warpPerspective(path_top, M_inv_, (w,h), flags=cv2.INTER_LINEAR)
    
    path_added_img = cv2.addWeighted(img, 1.0, unwarped_path, 0.9, 0)
    
    path_top2 = cv2.resize(path_top, (h,h), interpolation = cv2.INTER_AREA)    
#     path_added_img = cv2.hconcat([path_top2, path_added_img])
    sim_result_top_imgs.append(path_top_added)
    sim_result_imgs.append(path_added_img)
    

## Outdoor

In [ ]:
# Create Nav System
CONFIG_FILE = "cfg/outdoor_pspnet50_navigation.yaml"
cfg = config.load_cfg_from_cfg_file(CONFIG_FILE)
mtxs = np.load(cfg.perspective_transform_path)
M_ = mtxs['M']
M_inv_ = mtxs['M_inv']
nav = NavigationSystem(cfg)

# List of images
img_names = ['left0155.jpg',
'left0223.jpg',
'left0337.jpg',
'left0417.jpg',
'left0562.jpg',
'left0663.jpg',
'left0666.jpg',
'left0786.jpg',
'left1055.jpg',
'left1069.jpg',
'left1080.jpg',
'left1199.jpg',
'left1292.jpg',
'left1390.jpg',
'left1487.jpg'
            ]
out_segmented_imgs = []
out_top_imgs = []
out_result_imgs= []
out_result_top_imgs= []

for img_name in img_names:
    img = cv2.imread("./data/outdoor1/"+img_name)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    dim = (cfg.original_width, cfg.original_height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w,_ = img.shape
    path, result_img, result_top, segmented_img, drivable_img, cost_fwd, cost_obst, cost_center, cost, path_top_img, lines= nav.global_planner_step(img, None)
    
    # ======= Obtain visualizations ===========
    # Segmented
    drivable_added_img = cv2.addWeighted(img, 0.9, drivable_img, 0.8, 0) 
    out_segmented_imgs.append(drivable_added_img)
    
    # Top View
    drivable_added_top = cv2.warpPerspective(drivable_added_img, M_, (480, 480), flags=cv2.INTER_LINEAR)
    drivable_added_top2 = cv2.resize(drivable_added_top, (h,h), interpolation = cv2.INTER_AREA) 
    out_top_imgs.append(drivable_added_top)

    # Results
    colormap = cv2.COLORMAP_HOT
    cost_heatmap = cv2.applyColorMap(np.uint8(255*cost/np.amax(cost)), colormap)
    cost_heatmap = cv2.cvtColor(cost_heatmap, cv2.COLOR_BGR2RGB)

    cost_fwd_heatmap = cv2.applyColorMap(np.uint8(255*cost_fwd/np.amax(cost_fwd)), colormap)
    cost_fwd_heatmap = cv2.cvtColor(cost_fwd_heatmap, cv2.COLOR_BGR2RGB)

    cost_center_heatmap = cv2.applyColorMap(np.uint8(255*cost_center/np.amax(cost_center)), colormap)
    cost_center_heatmap = cv2.cvtColor(cost_center_heatmap, cv2.COLOR_BGR2RGB)

    cost_obst_heatmap = cv2.applyColorMap(np.uint8(255*cost_obst/np.amax(cost_obst)), colormap)
    cost_obst_heatmap = cv2.cvtColor(cost_obst_heatmap, cv2.COLOR_BGR2RGB)

    # Paths
    path_img = 0*cost_heatmap.copy()
    x_idxs = np.uint16(path[:,0]*cfg.pixel_per_meter_x+cfg.width/2)
    y_idxs = -np.uint16(path[:,1]*cfg.pixel_per_meter_y-cfg.height)
    for i in range(x_idxs.shape[-1]-1):
        path_img = cv2.line(path_img, (x_idxs[i],y_idxs[i]), (x_idxs[i+1],y_idxs[i+1]), (0, 255, 0), 3)  
    path_top = cv2.addWeighted(cost_obst_heatmap, 0.7, path_img, 2.0, 0) 
    
    edge_lines_img = 0*cost_heatmap.copy()
    if lines is not None:
        lines = np.array(lines, dtype = np.int)
        for line in lines:
            path_top = cv2.line(path_top, (line[0],line[1]), (line[2],line[3]), (50, 50, 255), 3)
    path_top_added = cv2.addWeighted(img_top, 0.8, path_top, 1.0, 0)
    
    cv2.circle(path_top, (x_idxs[cfg.look_ahead], y_idxs[cfg.look_ahead]),10, (255, 255, 255), -1)
    cv2.putText(path_top, 'L', (x_idxs[cfg.look_ahead]+15, y_idxs[cfg.look_ahead]+0), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (255, 255, 255), 2, cv2.LINE_AA)
    
    axes = cv2.imread("./results/axes.png")
    axes_img = cost_heatmap.copy()*0
    axes_img[480-62:480,(240-48-15):(240+48-15),:] = axes[0:62,:,:]
    # cost_heatmap = axes_img
    _,thresh1 = cv2.threshold(cv2.cvtColor(axes_img, cv2.COLOR_BGR2GRAY), 2,255,cv2.THRESH_BINARY_INV)
    thresh1 = np.uint8(thresh1/255)
    path_top[:,:,0]*=thresh1
    path_top[:,:,1]*=thresh1
    path_top[:,:,2]*=thresh1
    path_top+=cv2.cvtColor(axes_img, cv2.COLOR_BGR2RGB)

    
    path_top_added[:,:,0]*=thresh1
    path_top_added[:,:,1]*=thresh1
    path_top_added[:,:,2]*=thresh1
    path_top_added+=cv2.cvtColor(axes_img, cv2.COLOR_BGR2RGB)

    unwarped_path = cv2.warpPerspective(path_top, M_inv_, (w,h), flags=cv2.INTER_LINEAR)
    path_added_img = cv2.addWeighted(img, 0.8, unwarped_path, 1.0, 0)
    
    path_top2 = cv2.resize(path_top, (h,h), interpolation = cv2.INTER_AREA)    
#     path_added_img = cv2.hconcat([path_top2, path_added_img])
    out_result_top_imgs.append(path_top_added)
    out_result_imgs.append(path_added_img)
    

## Indoor

In [ ]:
# Create Nav System
CONFIG_FILE = "cfg/indoor_pspnet50_navigation.yaml"
cfg = config.load_cfg_from_cfg_file(CONFIG_FILE)
mtxs = np.load(cfg.perspective_transform_path)
M_ = mtxs['M']
M_inv_ = mtxs['M_inv']
nav = NavigationSystem(cfg)

# List of images
img_names = ['indoor1/left0117.jpg',
'indoor1/left0166.jpg',
'indoor1/left0176.jpg',
'indoor1/left0259.jpg',
'indoor1/left0263.jpg',
'indoor1/left0598.jpg',
'indoor1/left0677.jpg',
'indoor1/left0685.jpg',
'indoor1/left1487.jpg',
'indoor2/left0120.jpg',
'indoor2/left0135.jpg',
'indoor2/left0131.jpg',
'indoor2/left0257.jpg',
'indoor2/left0272.jpg',
'indoor2/left0440.jpg',
'indoor2/left0715.jpg']

in_segmented_imgs = []
in_top_imgs = []
in_result_imgs= []
in_result_top_imgs= []

for img_name in img_names:
    img = cv2.imread("./data/"+img_name)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    dim = (cfg.original_width, cfg.original_height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    path, result_img, result_top, segmented_img, drivable_img, cost_fwd, cost_obst, cost_center, cost, path_top_img, lines= nav.global_planner_step(img, None)
    
    # ======= Obtain visualizations ===========
    # Segmented
    drivable_added_img = cv2.addWeighted(img, 0.9, drivable_img, 0.8, 0) 
    in_segmented_imgs.append(drivable_added_img)
    
    # Top View
    drivable_added_top = cv2.warpPerspective(drivable_added_img, M_, (480, 480), flags=cv2.INTER_LINEAR)
    drivable_added_top2 = cv2.resize(drivable_added_top, (h,h), interpolation = cv2.INTER_AREA) 
    in_top_imgs.append(drivable_added_top)

    # Results
    colormap = cv2.COLORMAP_HOT
    cost_heatmap = cv2.applyColorMap(np.uint8(255*cost/np.amax(cost)), colormap)
    cost_heatmap = cv2.cvtColor(cost_heatmap, cv2.COLOR_BGR2RGB)

    cost_fwd_heatmap = cv2.applyColorMap(np.uint8(255*cost_fwd/np.amax(cost_fwd)), colormap)
    cost_fwd_heatmap = cv2.cvtColor(cost_fwd_heatmap, cv2.COLOR_BGR2RGB)

    cost_center_heatmap = cv2.applyColorMap(np.uint8(255*cost_center/np.amax(cost_center)), colormap)
    cost_center_heatmap = cv2.cvtColor(cost_center_heatmap, cv2.COLOR_BGR2RGB)

    cost_obst_heatmap = cv2.applyColorMap(np.uint8(255*cost_obst/np.amax(cost_obst)), colormap)
    cost_obst_heatmap = cv2.cvtColor(cost_obst_heatmap, cv2.COLOR_BGR2RGB)

    # Paths
    path_img = 0*cost_heatmap.copy()
    x_idxs = np.uint16(path[:,0]*cfg.pixel_per_meter_x+cfg.width/2)
    y_idxs = -np.uint16(path[:,1]*cfg.pixel_per_meter_y-cfg.height)
    for i in range(x_idxs.shape[-1]-1):
        path_img = cv2.line(path_img, (x_idxs[i],y_idxs[i]), (x_idxs[i+1],y_idxs[i+1]), (0, 255, 0), 3)  
    path_top = cv2.addWeighted(cost_obst_heatmap, 0.8, path_img, 2.0, 0) 
    
    edge_lines_img = 0*cost_heatmap.copy()
    lines = np.array(lines, dtype = np.int)
    for line in lines:
        path_top = cv2.line(path_top, (line[0],line[1]), (line[2],line[3]), (50, 50, 255), 3)
    path_top_added = cv2.addWeighted(img_top, 0.8, path_top, 0.9, 0)
    
    
    if(x_idxs.shape[-1] > cfg.look_ahead):
        ix = x_idxs[cfg.look_ahead]
        iy = y_idxs[cfg.look_ahead]
    else:
        ix = x_idxs[-1]
        iy = y_idxs[-1]
    cv2.circle(path_top, (ix, iy),10, (255, 255, 255), -1)
    cv2.putText(path_top, 'L', (ix+15, iy+0), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (255, 255, 255), 2, cv2.LINE_AA)
    axes = cv2.imread("./results/axes.png")
    axes_img = cost_heatmap.copy()*0
    axes_img[480-62:480,(240-48-15):(240+48-15),:] = axes[0:62,:,:]
    # cost_heatmap = axes_img
    _,thresh1 = cv2.threshold(cv2.cvtColor(axes_img, cv2.COLOR_BGR2GRAY), 2,255,cv2.THRESH_BINARY_INV)
    thresh1 = np.uint8(thresh1/255)
    path_top[:,:,0]*=thresh1
    path_top[:,:,1]*=thresh1
    path_top[:,:,2]*=thresh1
    path_top+=cv2.cvtColor(axes_img, cv2.COLOR_BGR2RGB)

    
    
    path_top_added[:,:,0]*=thresh1
    path_top_added[:,:,1]*=thresh1
    path_top_added[:,:,2]*=thresh1
    path_top_added+=cv2.cvtColor(axes_img, cv2.COLOR_BGR2RGB)

    unwarped_path = cv2.warpPerspective(path_top, M_inv_, (w,h), flags=cv2.INTER_LINEAR)
    path_added_img = cv2.addWeighted(img, 1.0, unwarped_path, 0.9, 0)
    
    path_top2 = cv2.resize(path_top, (h,h), interpolation = cv2.INTER_AREA)    
#     path_added_img = cv2.hconcat([path_top2, path_added_img])
    in_result_top_imgs.append(path_top_added)
    in_result_imgs.append(path_added_img)

# Figures
## Segmented images

In [ ]:
# Top view
import matplotlib
font = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 26}

matplotlib.rc('font', **font)

# # segmented imgs
out_idxs = [3,5,6,12,12]
in_idxs = [3,4,6,7,14]
sim_idxs = [1,4,5,7,14]

out_idxs = [9]
in_idxs = [13]
sim_idxs = [1]

f, axs = plt.subplots(1, 3,figsize=(20,26))
axs = [axs]
for i,ax in enumerate(axs):
    ax[0].imshow(out_segmented_imgs[out_idxs[i]])
    ax[0].axis('off')
    ax[0].set_title('Outdoor')
    ax[1].imshow(in_segmented_imgs[in_idxs[i]])
    ax[1].axis('off')
    ax[1].set_title('Indoor')
    sim_segmented_imgs[sim_idxs[i]] = cv2.resize(sim_segmented_imgs[sim_idxs[i]], dim, interpolation = cv2.INTER_AREA)
    ax[2].imshow(sim_segmented_imgs[sim_idxs[i]])
    ax[2].axis('off')
    ax[2].set_title('Simulation')
fig.tight_layout()
plt.savefig(results_path+'01_semantic.png',dpi=300)
plt.show()

## Top-view

In [ ]:


f, axs = plt.subplots(3, 2,figsize=(20,24),gridspec_kw={'width_ratios': [1.335, 1]})
axs[0][1].imshow(out_top_imgs[3])
axs[0][1].set_xlabel('Y[m]')
axs[0][1].set_ylabel('X[m]')
axs[0][1].set_title('Top-Down-View')
axs[0][1].set_xticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
axs[0][1].set_xticklabels(np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-96)/80.0,2))
axs[0][1].set_yticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
axs[0][1].set_yticklabels(np.around(np.arange((drivable_added_top.shape[0]+1), 0,-96)/90.0,1))
print(out_top_imgs[3].shape)
axs[0][0].imshow(out_segmented_imgs[3])
axs[0][0].axis('off')
axs[0][0].set_title('Drivable Area')




axs[1][1].imshow(in_top_imgs[4])
axs[1][1].set_xlabel('Y[m]')
axs[1][1].set_ylabel('X[m]')
axs[1][1].set_xticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
axs[1][1].set_xticklabels(np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-96)/80.0,2))
axs[1][1].set_yticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
axs[1][1].set_yticklabels(np.around(np.arange((drivable_added_top.shape[0]+1), 0,-96)/90.0,1))
axs[1][0].imshow(in_segmented_imgs[4])
axs[1][0].axis('off')

axs[2][1].imshow(sim_top_imgs[4])
axs[2][1].set_xlabel('Y[m]')
axs[2][1].set_ylabel('X[m]')
axs[2][1].set_xticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
axs[2][1].set_xticklabels(np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-96)/60.0,2))
axs[2][1].set_yticks(np.arange(0, drivable_added_top.shape[0]+1,96)) 
axs[2][1].set_yticklabels(np.around(np.arange((drivable_added_top.shape[0]+1), 0,-96)/70.0,1))
sim_segmented_imgs[4] = cv2.resize(sim_segmented_imgs[4], dim, interpolation = cv2.INTER_AREA)

axs[2][0].imshow(sim_segmented_imgs[4])
axs[2][0].axis('off')

f.tight_layout()
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.07, 
                    hspace=0.17)
fig.tight_layout()
plt.savefig(results_path+'01_perspective.png',dpi=300)

plt.show()

# Final Result

In [ ]:
font = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 28}

matplotlib.rc('font', **font)
# # segmented imgs
out_idxs = [3,6,13,4,5]
in_idxs = [13,6,7,14,4]
sim_idxs = [4,1,5,7,14]
# results

f, axs = plt.subplots(3, 5,figsize=(24,12))
for i in range(5):
    axs[0][i].imshow(out_result_imgs[out_idxs[i]])
    axs[0][i].set_xticks([]) 
    axs[0][i].set_yticks([]) 

for i in range(5):
    axs[1][i].imshow(in_result_imgs[in_idxs[i]])
    axs[1][i].set_xticks([]) 
    axs[1][i].set_yticks([]) 
    
for i in range(5):
    sim_result_imgs[sim_idxs[i]] = cv2.resize(sim_result_imgs[sim_idxs[i]], dim, interpolation = cv2.INTER_AREA)
    axs[2][i].imshow(sim_result_imgs[sim_idxs[i]])
    axs[2][i].set_xticks([]) 
    axs[2][i].set_yticks([]) 

axs[0][0].set_ylabel('Outdoor')
axs[1][0].set_ylabel('Indoor')
axs[2][0].set_ylabel('Simulation')

    
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.1)
fig.tight_layout()
plt.savefig(results_path+'01_results.png',dpi=300)

plt.show()

# f, axs = plt.subplots(5, 3,figsize=(28,35))
# for i,ax in enumerate(axs):
#     ax[0].imshow(out_result_imgs[out_idxs[i]])
#     ax[0].axis('off')
#     ax[1].imshow(in_result_imgs[in_idxs[i]])
#     ax[1].axis('off')
#     ax[2].imshow(sim_result_imgs[sim_idxs[i]])
#     ax[2].axis('off')
    
# plt.subplots_adjust(left=0.1,
#                     bottom=0.1, 
#                     right=0.9, 
#                     top=0.9, 
#                     wspace=0.1, 
#                     hspace=0.1)
# fig.tight_layout()
# plt.show()

In [ ]:
##### # results
f, axs = plt.subplots(15, 3,figsize=(28,110))
for i,ax in enumerate(axs):
    ax[0].imshow(out_result_imgs[i])
    ax[0].axis('off')
    ax[1].imshow(in_result_imgs[i])
    ax[1].axis('off')
    ax[2].imshow(sim_result_imgs[i])
    ax[2].axis('off')
    
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.1)
fig.tight_layout()
plt.show()

# Top view
f, axs = plt.subplots(3, 2,figsize=(20,18),gridspec_kw={'width_ratios': [1.335, 1]})
axs[0][1].imshow(out_top_imgs[3])
axs[0][1].set_xlabel('Y[m]')
axs[0][1].set_ylabel('X[m]')
# plt.xticks(np.arange(0, drivable_added_top.shape[0]+1,48), np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-48)/80.0,2))
# plt.yticks(np.arange(0, drivable_added_top.shape[0]+1,48), np.around(np.arange((drivable_added_top.shape[0]+1), 0,-48)/90.0,1))
axs[0][0].imshow(out_segmented_imgs[3])
axs[0][0].axis('off')

axs[1][1].imshow(in_top_imgs[4])
axs[1][1].set_xlabel('Y[m]')
axs[1][1].set_ylabel('X[m]')
# plt.xticks(np.arange(0, drivable_added_top.shape[0]+1,48), np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-48)/80.0,2))
# plt.yticks(np.arange(0, drivable_added_top.shape[0]+1,48), np.around(np.arange((drivable_added_top.shape[0]+1), 0,-48)/90.0,1))
axs[1][0].imshow(in_segmented_imgs[4])
axs[1][0].axis('off')

axs[2][1].imshow(sim_top_imgs[4])
axs[2][1].set_xlabel('Y[m]')
axs[2][1].set_ylabel('X[m]')
# plt.xticks(np.arange(0, drivable_added_top.shape[0]+1,48), np.around(np.arange(drivable_added_top.shape[0]/2, -(drivable_added_top.shape[0]+1)/2,-48)/80.0,2))
# plt.yticks(np.arange(0, drivable_added_top.shape[0]+1,48), np.around(np.arange((drivable_added_top.shape[0]+1), 0,-48)/90.0,1))
axs[2][0].imshow(sim_segmented_imgs[4])
axs[2][0].axis('off')

f.tight_layout()
plt.show()


# # segmented imgs
f, axs = plt.subplots(15, 3,figsize=(20,120))
for i,ax in enumerate(axs):
    ax[0].imshow(out_segmented_imgs[i])
    ax[0].axis('off')
    ax[1].imshow(in_segmented_imgs[i])
    ax[1].axis('off')
    ax[2].imshow(sim_segmented_imgs[i])
    ax[2].axis('off')
fig.tight_layout()
plt.show()


In [ ]:

# ==== Create visualizations ===== #
# Segmentation
drivable_added_img = cv2.addWeighted(img, 0.7, drivable_img, 0.9, 0)  
segmented_added_img = cv2.addWeighted(img, 1.0, segmented_img, 0.5, 0) 



# Perspective Transformation
img_top = cv2.warpPerspective(img, M_, (480, 480), flags=cv2.INTER_LINEAR)

drivable_top = cv2.warpPerspective(drivable_img, M_, (480, 480), flags=cv2.INTER_LINEAR)
drivable_added_top = cv2.warpPerspective(drivable_added_img, M_, (480, 480), flags=cv2.INTER_LINEAR)
segmented_added_top = cv2.warpPerspective(segmented_added_img, M_, (480, 480), flags=cv2.INTER_LINEAR)

drivable_top2 = cv2.resize(drivable_top, (h,h), interpolation = cv2.INTER_AREA)  
drivable_added_top2 = cv2.resize(drivable_added_top, (h,h), interpolation = cv2.INTER_AREA) 
segmented_added_top2 = cv2.resize(segmented_added_top, (h,h), interpolation = cv2.INTER_AREA)    


drivable_top_concat = cv2.hconcat([drivable_added_img, drivable_top2])
drivable_addedtop_concat = cv2.hconcat([drivable_added_img, drivable_added_top2])
segmented_addedtop_concat = cv2.hconcat([segmented_added_img, segmented_added_top2])



# Costmaps
colormap = cv2.COLORMAP_HOT
cost_heatmap = cv2.applyColorMap(np.uint8(255*cost/np.amax(cost)), colormap)
cost_heatmap = cv2.cvtColor(cost_heatmap, cv2.COLOR_BGR2RGB)

cost_fwd_heatmap = cv2.applyColorMap(np.uint8(255*cost_fwd/np.amax(cost_fwd)), colormap)
cost_fwd_heatmap = cv2.cvtColor(cost_fwd_heatmap, cv2.COLOR_BGR2RGB)

cost_center_heatmap = cv2.applyColorMap(np.uint8(255*cost_center/np.amax(cost_center)), colormap)
cost_center_heatmap = cv2.cvtColor(cost_center_heatmap, cv2.COLOR_BGR2RGB)

cost_obst_heatmap = cv2.applyColorMap(np.uint8(255*cost_obst/np.amax(cost_obst)), colormap)
cost_obst_heatmap = cv2.cvtColor(cost_obst_heatmap, cv2.COLOR_BGR2RGB)

# Paths
path_img = 0*cost_heatmap.copy()
x_idxs = np.uint16(path[:,0]*cfg.pixel_per_meter_x+cfg.width/2)
y_idxs = -np.uint16(path[:,1]*cfg.pixel_per_meter_y-cfg.height)
for i in range(x_idxs.shape[-1]-1):
    path_img = cv2.line(path_img, (x_idxs[i],y_idxs[i]), (x_idxs[i+1],y_idxs[i+1]), (50, 255, 50), 3)  
path_top = cv2.addWeighted(cost_obst_heatmap, 1.0, path_img, 0.7, 0) 
edge_lines_img = 0*cost_heatmap.copy()
lines = np.array(lines, dtype = np.int)
for line in lines:
    path_top = cv2.line(path_top, (line[0],line[1]), (line[2],line[3]), (50, 50, 255), 3)
path_top_added = cv2.addWeighted(img_top, 0.8, path_top, 1.0, 0)

unwarped_path = cv2.warpPerspective(path_top, M_inv_, (w,h), flags=cv2.INTER_LINEAR)
path_added_img = cv2.addWeighted(img, 0.8, unwarped_path, 1.0, 0)

path_top2 = cv2.resize(path_top, (h,h), interpolation = cv2.INTER_AREA)    
path_added_img = cv2.hconcat([path_top2, path_added_img])


# x_idxs = path[:,0]*cfg.pixel_per_meter_x+cfg.width/2
# y_idxs = -(path[:,1]*cfg.pixel_per_meter_y-cfg.height)
# ax.plot(x_idxs,y_idxs, color='greenyellow', linewidth=4)

#seg
plt.imshow(segmented_added_img)
plt.show()

#perspective
plt.imshow(drivable_top_concat)
plt.show()
plt.imshow(drivable_addedtop_concat)
plt.show()
plt.imshow(segmented_addedtop_concat)
plt.show()

#cost
plt.imshow(cost_obst_heatmap)
plt.show()
plt.imshow(cost_fwd_heatmap)
plt.show()
plt.imshow(cost_center_heatmap)
plt.show()
plt.imshow(cost_heatmap)
plt.show()

#path
plt.imshow(path_top_added)
plt.show()

plt.imshow(path_added_img)
plt.show()

In [ ]:
imgs, filenames = load_images_from_folder('./data/indoor2/')
print('processing {} images'.format(len(imgs)))
i=0
start = time.time()

for img, filename in zip(imgs, filenames):
    print('===============File: {} ============'.format(filename))
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    path, result_img, result_birdview, segmented_img = nav.global_planner_step(img, None)
    
    h,w,_ = result_img.shape
    result_birdview = cv2.resize(result_birdview, (h,h), interpolation = cv2.INTER_AREA)
    segmented_img = cv2.resize(segmented_img, (w,h), interpolation = cv2.INTER_AREA)
    
    concat = cv2.hconcat([segmented_img, result_img, result_birdview])
#     fig, ax = plt.subplots(figsize=(14, 7))
#     ax.imshow(concat)
#     plt.show()   
    concat = cv2.cvtColor(concat, cv2.COLOR_BGR2RGB)

    cv2.imwrite('./output/indoor2/{}'.format(filename), concat)
    i = i+1
end = time.time()
print('avg time: {}'.format((end-start)/len(imgs)))

In [ ]:
imgs, filenames = load_images_from_folder('./data/indoor1/')
print('processing {} images'.format(len(imgs)))
i=0
start = time.time()

for img, filename in zip(imgs, filenames):
    print('===============File: {} ============'.format(filename))
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    path, result_img, result_birdview, segmented_img = nav.global_planner_step(img, None)
    
    h,w,_ = result_img.shape
    result_birdview = cv2.resize(result_birdview, (h,h), interpolation = cv2.INTER_AREA)
    segmented_img = cv2.resize(segmented_img, (w,h), interpolation = cv2.INTER_AREA)
    
    concat = cv2.hconcat([segmented_img, result_img, result_birdview])
#     fig, ax = plt.subplots(figsize=(14, 7))
#     ax.imshow(concat)
#     plt.show()   
    concat = cv2.cvtColor(concat, cv2.COLOR_BGR2RGB)

    cv2.imwrite('./output/indoor1/{}'.format(filename), concat)
    i = i+1
end = time.time()
print('avg time: {}'.format((end-start)/len(imgs)))

In [ ]:
CONFIG_FILE = "cfg/indoor_pspnet50_navigation.yaml"
cfg = config.load_cfg_from_cfg_file(CONFIG_FILE)
nav = NavigationSystem(cfg)
imgs, filenames = load_images_from_folder('./data/indoor1/')
print('processing {} images'.format(len(imgs)))
i=0
start = time.time()

for img, filename in zip(imgs, filenames):
    print('===============File: {} ============'.format(filename))
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    path, result_img, result_birdview, segmented_img = nav.global_planner_step(img, None)
    
    h,w,_ = result_img.shape
    result_birdview = cv2.resize(result_birdview, (h,h), interpolation = cv2.INTER_AREA)
    segmented_img = cv2.resize(segmented_img, (w,h), interpolation = cv2.INTER_AREA)
    
    concat = cv2.hconcat([segmented_img, result_img, result_birdview])
#     fig, ax = plt.subplots(figsize=(14, 7))
#     ax.imshow(concat)
#     plt.show()   
    concat = cv2.cvtColor(concat, cv2.COLOR_BGR2RGB)

    cv2.imwrite('./output/indoor1/{}'.format(filename), concat)
    i = i+1
end = time.time()
print('avg time: {}'.format((end-start)/len(imgs)))

In [ ]:
imgs, filenames = load_images_from_folder('./data/indoor2/')
print('processing {} images'.format(len(imgs)))
i=0
start = time.time()

for img, filename in zip(imgs, filenames):
    print('===============File: {} ============'.format(filename))
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    path, result_img, result_birdview, segmented_img = nav.global_planner_step(img, None)
    
    h,w,_ = result_img.shape
    result_birdview = cv2.resize(result_birdview, (h,h), interpolation = cv2.INTER_AREA)
    segmented_img = cv2.resize(segmented_img, (w,h), interpolation = cv2.INTER_AREA)
    
    concat = cv2.hconcat([segmented_img, result_img, result_birdview])
#     fig, ax = plt.subplots(figsize=(14, 7))
#     ax.imshow(concat)
#     plt.show()   
    concat = cv2.cvtColor(concat, cv2.COLOR_BGR2RGB)

    cv2.imwrite('./output/indoor2/{}'.format(filename), concat)
    i = i+1
end = time.time()
print('avg time: {}'.format((end-start)/len(imgs)))

In [ ]:
imgs, filenames = load_images_from_folder('./data/indoor3/')
print('processing {} images'.format(len(imgs)))
i=0
start = time.time()

for img, filename in zip(imgs, filenames):
    print('===============File: {} ============'.format(filename))
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    path, result_img, result_birdview, segmented_img = nav.global_planner_step(img, None)
    
    h,w,_ = result_img.shape
    result_birdview = cv2.resize(result_birdview, (h,h), interpolation = cv2.INTER_AREA)
    segmented_img = cv2.resize(segmented_img, (w,h), interpolation = cv2.INTER_AREA)
    
    concat = cv2.hconcat([segmented_img, result_img, result_birdview])
#     fig, ax = plt.subplots(figsize=(14, 7))
#     ax.imshow(concat)
#     plt.show()   
    concat = cv2.cvtColor(concat, cv2.COLOR_BGR2RGB)

    cv2.imwrite('./output/indoor3/{}'.format(filename), concat)
    i = i+1
end = time.time()
print('avg time: {}'.format((end-start)/len(imgs)))

In [ ]:
imgs, filenames = load_images_from_folder('./data/jackal_cam/jackal_cam_indoors1')
print('processing {} images'.format(len(imgs)))
i=0
start = time.time()

for img, filename in zip(imgs, filenames):
    print('===============File: {} ============'.format(filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    path, result_img, result_birdview, segmented_img = nav.global_planner_step(img, None)
    
    h,w,_ = result_img.shape
    result_birdview = cv2.resize(result_birdview, (h,h), interpolation = cv2.INTER_AREA)
    segmented_img = cv2.resize(segmented_img, (w,h), interpolation = cv2.INTER_AREA)
    
    concat = cv2.hconcat([segmented_img, result_img, result_birdview])
#     fig, ax = plt.subplots(figsize=(14, 7))
#     ax.imshow(concat)
#     plt.show()   
    concat = cv2.cvtColor(concat, cv2.COLOR_BGR2RGB)

    cv2.imwrite('./output/out/{}'.format(filename), concat)
    i = i+1
end = time.time()
print('avg time: {}'.format((end-start)/len(imgs)))

In [ ]:

import torch
torch.backends.cudnn.benchmark = True
import argparse
import os
import cv2
import sys
import numpy as np
from object_detection import ObjectDetector
from fchardnet_segmentation import FCHarDNetSemanticSegmentation
from helpers import warped2scan, warp_driveable, get_driveable_mask,get_driveable_mask2
import math
from collections import deque
%matplotlib inline
FORWARD_WEIGHT = 3.0
CENTER_WEIGHT = 0.5
HEIGHT=480
WIDTH=480
OSCILLATIONS_DETECTION_LENGTH = 3
PIXEL_PER_METER_X = (WIDTH - 2*150)/3.0 #Horizontal distance between src points in the real world ( I assumed 3.0 meters)
PIXEL_PER_METER_Y = (HEIGHT - 20-55)/6.0 #Vertical distance between src points in the real world ( I assumed 6.0 meters)
HORIZ_ANGLE_THRESHOLD = 15*math.pi/180.0
AVG_SIDEWALK_WIDTH = round(3.9*PIXEL_PER_METER_X)


class PerceptionSystem(object):
    def __init__(self): 
        # Load Models
        segmentation_model_path = os.path.join('/usr/src/app/dev_ws/src/vision/vision', 'pretrained', 'hardnet70_cityscapes_model.pkl')
        self.seg_model_ = FCHarDNetSemanticSegmentation(segmentation_model_path)
        self.object_detector_ = ObjectDetector()
        
        # Load perspective transforms
        mtxs = np.load('/usr/src/app/dev_ws/src/vision/vision/PerspectiveTransform.npz')
        self.M_ = mtxs['M']
        self.M_inv_ = mtxs['M_inv']
        
        # Test Detection Models
        print('Segmentation and Detection Models loaded, Testing the models')
        img = cv2.imread("./data/73.png")
        self.h_orig_, self.w_orig_,_ = img.shape
        _, _ = self.seg_model_.process_img_driveable(img,[self.h_orig_,self.w_orig_])
        _ = self.object_detector_.process_frame(img)
        self.im_hw_ = self.object_detector_.im_hw
        print('Imgs tested')
        
    def get_driveable(self, driveable_decoded):
        h,w,_ = driveable_decoded.shape
        # Warp driveable area
        warped = cv2.warpPerspective(driveable_decoded, self.M_, (480, 480), flags=cv2.INTER_LINEAR)
        # Calculate robot center
        original_center = np.array([[[w/2,h]]],dtype=np.float32)
        warped_center = cv2.perspectiveTransform(original_center, self.M_)[0][0]    
        driveable_contour_mask = get_driveable_mask2(warped, warped_center)
        return driveable_contour_mask
    
    def add_detections_birdview(self, preds, driveable_mask):
        h,w,_ = self.object_detector_.im_hw
        h_rate = self.h_orig_/h
        w_rate = self.w_orig_/w
        for pred in preds:
            if(pred[4] > self.object_detector_.conf_thres): # If prediction has a bigger confidence than the threshold
                x = w_rate*(pred[0]+pred[2])/2.0 # Ground middle point
                y = h_rate*pred[3]
                if(pred[5]==0): #person
                    wr = 20
                    hr = 20
                    color = 150
                else:
                    wr = 20
                    hr = 40
                    color = 150
                pos_orig = np.array([[[x,y]]],dtype=np.float32)
                warped_birdview = cv2.perspectiveTransform(pos_orig, self.M_)[0][0] # Transform middle ground point to birdview
                warped_birdview = np.uint16(warped_birdview)
                cv2.rectangle(driveable_mask, (warped_birdview[0] -int(wr/2), warped_birdview[1]-int(hr/2)), (warped_birdview[0] +int(wr/2), warped_birdview[1]+int(hr/2)), color, -1) 

        
    def process_frame(self,img):
        # Semantic Segmentation
        img_test = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_decoded, driveable_decoded = self.seg_model_.process_img_driveable(img_test,[self.h_orig_,self.w_orig_])
        
        # Get bird eye view with driveable area limits
        driveable_mask  = self.get_driveable(driveable_decoded)
        
        # Object Detection
        preds = self.object_detector_.process_frame(img)
        
        # Add Detections to birdview image
        driveable_mask_with_objects = driveable_mask.copy()
        self.add_detections_birdview(preds, driveable_mask_with_objects)
        
        return driveable_mask, preds, driveable_mask_with_objects
        
class CostMap(object):
    def __init__(self, M):
        # Temporary
        self.M_ = M
        self.h_orig_ = 720
        self.w_orig_ = 1080
        
    def calculate_costmap(self,driveable_mask, preds, driveable_mask_with_objects):
        
        
        # Find sidewalk edge lines and angle
        # h,w = driveable_mask.shape
        angle_avg, m_avg, b_avg = self.sidewalk_lines(driveable_mask, driveable_mask_with_objects) #driveable_mask_with_objects contains all objects and lines
        
        ## Find distance to center cost
        cost_center = self.center_cost(m_avg,b_avg)

        # Create obstacle cost map
        cost_obst = self.obstacle_cost(driveable_mask_with_objects)

        # Create inclination plane (forward cost)
        cost_forward = self.forward_cost(angle_avg)

        # Total cost
        cost_fcn = cost_obst+cost_forward*FORWARD_WEIGHT+cost_center*CENTER_WEIGHT
        
        return cost_fcn, cost_obst
        

    def gaus2d(self, x=0, y=0, mx=0, my=0, sx=1, sy=1):
        return 1. / (2. * np.pi * sx * sy) * np.exp(-((x - mx)**2. / (2. * sx**2.) + (y - my)**2. / (2. * sy**2.)))
    
    def obstacle_cost(self, mask_with_objects, gaussian_shape = 150):
        x = np.linspace(-2, 2,gaussian_shape)
        y = np.linspace(-2, 2,gaussian_shape*2)
        x, y = np.meshgrid(x, y) # get 2D variables instead of 1D
        z = self.gaus2d(x, y)
        z = np.float32(z)
        cost_obst = cv2.filter2D(mask_with_objects,-1,z)
        cost_obst/=np.amax(cost_obst)
        return cost_obst

    def forward_cost(self, angle):
        normal = np.array([math.tan(-angle),-1,HEIGHT])
        point = np.array([WIDTH/2, HEIGHT, 0])
        d = -np.sum(point*normal)# dot product
        xx, yy = np.meshgrid(range(WIDTH), range(HEIGHT))
        cost_forward = (-normal[0]*xx - normal[1]*yy - d)*1./normal[2]
        return cost_forward

    def center_cost(self, m,b):
        if m is not None:
            xx, yy = np.meshgrid(range(WIDTH), range(HEIGHT))
            cost_center = abs(-m*xx+yy-b)/math.sqrt(m**2+1)
        else:
            cost_center = np.zeros((WIDTH,HEIGHT))
        return cost_center/np.amax(cost_center)


   
    def sidewalk_lines(self, mask, mask_out):
        mask = np.uint8(mask)
        
        # Detect lines in driveable area mask
        lines = cv2.HoughLinesP(mask, 1, 1*np.pi / 180, 100, None, 150, 50)

        line_angles = []
        lines_left = []
        lines_right = []
        lines_found = True
        if(lines is not None):
            for line in lines:
                x2,y2,x1,y1 = line[0]
                # Calculate angle, checking which y-coordinate is higher
                if(y2<y1): angle = -math.atan2(y2-y1,x2-x1)-math.pi/2
                else: angle = -math.atan2(y1-y2,x1-x2)-math.pi/2
                line_angles.append(angle)
                    
                # Detect horizontal lines corresponding to the corners
                if(abs(angle-math.pi/2) < HORIZ_ANGLE_THRESHOLD):
                    cv2.line(mask_out,(round(x1),round(y1)),(round(x2),round(y2)),100,1)

                # Detect coordinate at the bottom of image
                if(x2 == x1): x2 += 1 # Avoid dividing by 0
                if(y2 == y1): y2 += 1 # Avoid dividing by 0
                    
                m = (y2-y1)/(x2-x1)
                b = y1-m*x1
                x3 = round((HEIGHT-b)/m)
                x1 = x3
                y1 = HEIGHT

                # Add lines to the left and right list
                if(x3<WIDTH/2):
                    lines_left.append([m,b,x2,y2,x1,y1])
                else:
                    lines_right.append([m,b,x2,y2,x1,y1])
    #         print('lines_left: {}'.format(np.array(lines_left)))
    #         print('lines_right: {}'.format(np.array(lines_right)))
            lines_left = np.array(lines_left)
            lines_right = np.array(lines_right)

            # Find average line left and right
            if(len(lines_left)>0):
                m_avg_left = np.average(lines_left[:,0])
                b_avg_left = np.average(lines_left[:,1])
                y1_left = np.amin(lines_left[:,3])
                x1_left = (y1_left - b_avg_left) / m_avg_left
                x2_left = (HEIGHT-b_avg_left) / m_avg_left
                if x2_left > 0 and x2_left < WIDTH:
                    y2_left = HEIGHT
                else:
                    x2_left = 0
                    y2_left = b_avg_left
                    
                if (len(lines_right) > 0): # both lines are found
                    m_avg_right = np.average(lines_right[:,0])
                    b_avg_right = np.average(lines_right[:,1])
                    y1_right = np.amin(lines_right[:,3])
                    x1_right = (y1_right - b_avg_right) / m_avg_right
                    x2_right = (HEIGHT-b_avg_right) / m_avg_right
                    if x2_right > 0 and x2_left < WIDTH:
                        y2_right = HEIGHT
                    else:
                        x2_right = 0
                        y2_right = b_avg_right
                else:
                    print('right line not found, adding it')
                    m_avg_right = m_avg_left  
                    b_avg_right = AVG_SIDEWALK_WIDTH*math.sqrt(m_avg_right**2+1)+b_avg_left
                    if((b_avg_right - b_avg_left) < 0):
                        b_avg_right = -AVG_SIDEWALK_WIDTH*math.sqrt(m_avg_right**2+1)+b_avg_left
                    y1_right = y1_left
                    x1_right = (y1_right - b_avg_right) / m_avg_right
                    y2_right = y2_left
                    x2_right = (y2_right - b_avg_right) / m_avg_right
            else:
                if (len(lines_right) > 0): # only right line found
                        
                    m_avg_right = np.average(lines_right[:,0])
                    b_avg_right = np.average(lines_right[:,1])
                    y1_right = np.amin(lines_right[:,3])
                    x1_right = (y1_right - b_avg_right) / m_avg_right
                    x2_right = (HEIGHT-b_avg_right) / m_avg_right
                    if x2_right > 0 and x2_right < WIDTH:
                        y2_right = HEIGHT
                    else:
                        x2_right = 0
                        y2_right = b_avg_right
                    m_avg_left = m_avg_right  
                    b_avg_left = -AVG_SIDEWALK_WIDTH*math.sqrt(m_avg_left**2+1)+b_avg_right
                    if((b_avg_left - b_avg_right) < 0):
        #                 print("si")
                        b_avg_left = AVG_SIDEWALK_WIDTH*math.sqrt(m_avg_right**2+1)+b_avg_right
                    y1_left = y1_right
                    x1_left = (y1_left - b_avg_left) / m_avg_left
                    y2_left = y2_right
                    x2_left = (y2_left - b_avg_left) / m_avg_left
                else:
                    angle_avg = 0
                    lines_found = False
        else:
            lines_found = False
        angle_avg = np.average(line_angles)

        if(lines_found):
    #         print([x1_right, y1_right, x2_right, y2_right])
    #         print([x1_left, y1_left, x2_left, y2_left])
            angle_left = -math.atan2(y2_left-y1_left,x2_left-x1_left)-math.pi/2
            angle_right = -math.atan2(y2_right-y1_right,x2_right-x1_right)-math.pi/2

            cv2.line(mask_out,(round(x1_left),round(y1_left)),(round(x2_left),round(y2_left)),200,3)
            cv2.line(mask_out,(round(x1_right),round(y1_right)),(round(x2_right),round(y2_right)),200,3)
            # Calculate middle line
            angle_avg = (angle_left + angle_right) / 2.0
            m_avg = math.tan(-angle_avg+math.pi/2.0)
            x_center = ( x2_left + x2_right ) / 2.0
            b_avg = HEIGHT - m_avg*x_center
        else:
            m_avg = None
            b_avg = None
        return angle_avg, m_avg, b_avg
                          

class PotentialFieldPlanner(object):
                          
    def __init__(self, M_inv):
        self.M_inv_ = M_inv
        print("Planner")
                         
    def get_motion_model(self):
        # dx, dy
        motion = [
    #               [1, 0], #right
    #               [0, 2], #back
    #               [-1, 0], #left
                  [0, -4], # front
                  [-1, -4],#front-left
                  [-2, -4],#front-left
                  [-3, -4],#front-left
                  [-4, -4],#front-left
    #               [-1, 2], # back-left
                  [1, -4], #front-right
                  [2, -4], #front-right
                  [3, -4], #front-right
                  [4, -4], #front-right
    #               [1, 1]  #back-right
                  ]
        return motion

    def oscillations_detection(self,previous_ids, ix, iy):
        previous_ids.append((ix, iy))
        if (len(previous_ids) > OSCILLATIONS_DETECTION_LENGTH):
            previous_ids.popleft()
        # check if contains any duplicates by copying into a set
        previous_ids_set = set()
        for index in previous_ids:
            if index in previous_ids_set:
                return True
            else:
                previous_ids_set.add(index)
        return False

    def calculate_path(self,pmap):
        output = pmap.copy()*0
        motion = self.get_motion_model()
        previous_ids = deque()
        ix = round(WIDTH/2)
        iy = round(HEIGHT) 
        path = []
        while(ix > 5 and iy > 50 and ix< WIDTH-5):
            minp = float("inf")
            minix, miniy = 0, -1
            for i, _ in enumerate(motion):
                inx = int(ix + 1*motion[i][0])
                iny = int(iy + 1*motion[i][1])
                if inx >= WIDTH or iny >= HEIGHT or inx < 0 or iny < 0:
                    p = float("inf")  # outside area
                    print("outside potential!")
                else:
                    p = pmap[iny][inx]

                if minp >= p:
                    min_motion = motion[i]
                    minp = p
                    minix = inx
                    miniy = iny
            ix = minix
            iy = miniy
            # Calculate points
            px = (ix-WIDTH/2)/PIXEL_PER_METER_X
            py = (HEIGHT-iy)/PIXEL_PER_METER_Y
            path.append([px,py])
            if (self.oscillations_detection(previous_ids, ix, iy)):
                print("Oscillation detected at ({},{})!".format(ix, iy))
                break
            cv2.circle(output, (ix, iy),int(3.0),1, -1)
        return np.array(path),output

    def draw_result(self, img, cost_obst, path_img):
        h_orig,w_orig,_ = img.shape

        result_birdview = cv2.merge([cost_obst, path_img, cost_obst*0])
        result_birdview = np.uint8(result_birdview*255.0)
        unwarped_birdview = cv2.warpPerspective(result_birdview, self.M_inv_, (w_orig,h_orig), flags=cv2.INTER_LINEAR)
    #     unwarped_birdview = np.uint8(unwarped_birdview*255.0)
        output = cv2.addWeighted(img, 0.7, unwarped_birdview, 0.5, 0)    
        return output, result_birdview

class NavigationSystem(object):
    def __init__(self): 
        self.perception_ = PerceptionSystem()
        self.costmap_ = CostMap(self.perception_.M_)
        self.planner_ = PotentialFieldPlanner(self.perception_.M_inv_)

    def path_planning(self, img):
        driveable_mask, preds, driveable_mask_with_objects = self.perception_.process_frame(img)
        cost,cost_obst = self.costmap_.calculate_costmap(driveable_mask, preds, driveable_mask_with_objects)
        path, path_img = self.planner_.calculate_path(cost)
        result_img, result_birdview = self.planner_.draw_result(img, cost_obst, path_img)
        fig, ax = plt.subplots(figsize=(20, 10))
        ax.imshow(result_img)
        plt.show()
#     def seg2scan(self, driveable_area):
#         h,w,_ = driveable_area.shape
#         warped = cv2.warpPerspective(driveable_area, M, (480, 480), flags=cv2.INTER_LINEAR)
#         original_center = np.array([[[w/2,h]]],dtype=np.float32)
#         warped_center = cv2.perspectiveTransform(original_center, M)[0][0]
#         scan_distances, angle_increment, warped_contours = warped2scan(warped, warped_center)
#         return warped, warped_contours, scan_distances, angle_increment

In [ ]:
import time
nav = NavigationSystem()
img = cv2.imread("./data/75.png")
# img = cv2.imread("./data/195.png")
h_orig,w_orig,_ = img.shape


img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
start = time.time()
nav.path_planning(img)
end = time.time()
print(end-start)
